# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-db366238e9-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Players explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sport. Compare players per countries, awards of the players and participation in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football player

2. Identify the BGP for association football club

3. Identify the BGP for association football league

4. Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

5. Identify the BGP for the ballon d'Or award

6. Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

7. Identify the BGP of the FIFA World Cup

8. Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

9. Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).


## Task 1
Identify the BGP for association football player

In [14]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?obj ?p.
   # get the label
   ?p sc:name ?name.
   #FILTER(REGEX(?name,"Profession"^^xsd:string))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1750244'), ('name', 'FWA Footballer of the Year')]
[('p', 'http://www.wikidata.org/entity/Q10726649'), ('name', "PFA Fans' Player of the Year")]
[('p', 'http://www.wikidata.org/entity/Q182994'), ('name', 'Liga Portugal')]
[('p', 'http://www.wikidata.org/entity/Q12899500'), ('name', 'Cristiano')]
[('p', 'http://www.wikidata.org/entity/Q20888805'), ('name', 'Commander of the Order of Merit of Portugal')]
[('p', 'http://www.wikidata.org/entity/Q3590758'), ('name', 'Portugal Olympic football team')]
[('p', 'http://www.wikidata.org/entity/Q373501'), ('name', 'UEFA Euro 2020')]
[('p', 'http://www.wikidata.org/entity/Q20819738'), ('name', 'dos Santos')]
[('p', 'http://www.wikidata.org/entity/Q1146578'), ('name', 'UEFA Team of the Year')]
[('p', 'http://www.wikidata.org/entity/Q182529'), ('name', 'FIFA World Player of the Year')]
[('p', 'http://www.wikidata.org/entity/Q260117'), ('name', "UEFA Men's Player of the Year Award")]
[('p', 'http://www.

In [15]:
#[('p', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q937857 ?o ?p .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P2369'), ('name', 'Soccerway player ID')]
[('p', 'http://www.wikidata.org/entity/P3908'), ('name', 'Reprezentacija player ID')]
[('p', 'http://www.wikidata.org/entity/Q18536342'), ('name', 'competitive player')]
[('p', 'http://www.wikidata.org/entity/P2446'), ('name', 'Transfermarkt player ID')]
[('p', 'http://www.wikidata.org/entity/Q7217464'), ('name', 'Category:Association football players')]
[('p', 'http://www.wikidata.org/entity/P2823'), ('name', 'Royal Belgian Football Association player ID')]
[('p', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('p', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football")]
[('p', 'http://www.wi

In [16]:
#[('p', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
#[('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q476028 ?o ?p .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q51481377'), ('name', "women's association football club")]
[('p', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/entity/Q628099'), ('name', 'association football manager')]
[('p', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
[('p', 'http://www.wikidata.org/entity/Q714581'), ('name', 'Nea Salamis Famagusta FC')]
[('p', 'http://www.wikidata.org/entity/Q167270'), ('name', 'trademark')]
[('p', 'http://www.wikidata.org/entity/Q17270000'), ('name', 'football club')]
[('p', 'http://www.wikidata.org/entity/Q28083137'), ('name', 'football team')]
[('p', 'http://www.wikidata.org/entity/P5424'), ('name', 'Lega Serie A team ID')]
[('p', 'http://www.wikidata.org/entity/Q5614332'), ('name', 'Category:Association football clubs')]
[('p', 'http://www.wikidata.org/enti

In [40]:
#[('p', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
#[('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
#[('p', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q937857 ?o ?p .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P2369'), ('name', 'Soccerway player ID')]
[('p', 'http://www.wikidata.org/entity/P3908'), ('name', 'Reprezentacija player ID')]
[('p', 'http://www.wikidata.org/entity/Q18536342'), ('name', 'competitive player')]
[('p', 'http://www.wikidata.org/entity/P2446'), ('name', 'Transfermarkt player ID')]
[('p', 'http://www.wikidata.org/entity/Q7217464'), ('name', 'Category:Association football players')]
[('p', 'http://www.wikidata.org/entity/P2823'), ('name', 'Royal Belgian Football Association player ID')]
[('p', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('p', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football")]
[('p', 'http://www.wi

Final query for this task

In [134]:
# write your final query
#[('p', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
#[('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
#[('p', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q937857 ?o ?p.
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P2369'), ('name', 'Soccerway player ID')]
[('p', 'http://www.wikidata.org/entity/P3908'), ('name', 'Reprezentacija player ID')]
[('p', 'http://www.wikidata.org/entity/Q18536342'), ('name', 'competitive player')]
[('p', 'http://www.wikidata.org/entity/P2446'), ('name', 'Transfermarkt player ID')]
[('p', 'http://www.wikidata.org/entity/Q7217464'), ('name', 'Category:Association football players')]
[('p', 'http://www.wikidata.org/entity/P2823'), ('name', 'Royal Belgian Football Association player ID')]
[('p', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('p', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football")]
[('p', 'http://www.wi

## Task 2
Identify the BGP for association football club

In [38]:
# [('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q476028 ?p ?o .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Q

Final query for this task

In [20]:
# got association football club in task 1 results
# [('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q476028 ?p ?o .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Q

## Task 3
Identify the BGP for association football league

In [93]:
# [('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
#[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
#[('p', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
#('o', 'http://www.wikidata.org/entity/Q2186316'), ('nm', 'Belgian Provincial leagues')]


queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
      wd:Q2186316 ?o ?p  .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?nm.
   #filter(regex(?nm,"league"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('o', 'http://www.wikidata.org/prop/direct/P17'), ('nm', 'country')]
[('p', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league'), ('o', 'http://www.wikidata.org/prop/direct/P31'), ('nm', 'instance of')]
[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football'), ('o', 'http://www.wikidata.org/prop/direct/P641'), ('nm', 'sport')]
[('p', 'http://www.wikidata.org/entity/Q381124'), ('name', 'Royal Belgian Football Association'), ('o', 'http://www.wikidata.org/prop/direct/P664'), ('nm', 'organizer')]
4


Final query for this task

In [94]:
#here it is BGP assosiation football leagues.
#[('p', 'http://www.wikidata.org/entity/Q15991303'),('name', 'association football league'), ('o', 'http://www.wikidata.org/prop/direct/P31'), ('nm', 'instance of')]
# [('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
#[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
#[('p', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
#('o', 'http://www.wikidata.org/entity/Q2186316'), ('nm', 'Belgian Provincial leagues')]


queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
      wd:Q15991303 ?o ?p  .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?nm.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q8085406'), ('name', 'Template:Infobox football league'), ('o', 'http://www.wikidata.org/prop/direct/P1424'), ('nm', "topic's main template")]
[('p', 'http://www.wikidata.org/entity/Q26887310'), ('name', 'association football team season'), ('o', 'http://www.wikidata.org/prop/direct/P1889'), ('nm', 'different from')]
[('p', 'http://www.wikidata.org/entity/P1132'), ('name', 'number of participants'), ('o', 'http://www.wikidata.org/prop/direct/P1963'), ('nm', 'properties for this type')]
[('p', 'http://www.wikidata.org/entity/P2500'), ('name', 'league level below'), ('o', 'http://www.wikidata.org/prop/direct/P1963'), ('nm', 'properties for this type')]
[('p', 'http://www.wikidata.org/entity/P6118'), ('name', 'season ends'), ('o', 'http://www.wikidata.org/prop/direct/P1963'), ('nm', 'properties for this type')]
[('p', 'http://www.wikidata.org/entity/P3195'), ('name', 'league points system'), ('o', 'http://www.wikidata.org/prop/direct/P1963'),

## Task 4
Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

In [114]:
#[('p', 'http://www.wikidata.org/entity/Q15991303'),('name', 'association football league'),
queryString = """
SELECT DISTINCT (count(?p) as ?NumOfLeagues) ?o ?nm
#it will total number of leagues in each country
WHERE {
   # bind something
   ?p ?i wd:Q15991303.
   ?p wdt:P17 ?o.
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?nm.
   
}
GROUP BY ?o ?nm
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfLeagues', '1'), ('o', 'http://www.wikidata.org/entity/Q1044'), ('nm', 'Sierra Leone')]
[('NumOfLeagues', '4'), ('o', 'http://www.wikidata.org/entity/Q215'), ('nm', 'Slovenia')]
[('NumOfLeagues', '49'), ('o', 'http://www.wikidata.org/entity/Q218'), ('nm', 'Romania')]
[('NumOfLeagues', '3'), ('o', 'http://www.wikidata.org/entity/Q222'), ('nm', 'Albania')]
[('NumOfLeagues', '3'), ('o', 'http://www.wikidata.org/entity/Q232'), ('nm', 'Kazakhstan')]
[('NumOfLeagues', '5'), ('o', 'http://www.wikidata.org/entity/Q28'), ('nm', 'Hungary')]
[('NumOfLeagues', '1'), ('o', 'http://www.wikidata.org/entity/Q424'), ('nm', 'Cambodia')]
[('NumOfLeagues', '2'), ('o', 'http://www.wikidata.org/entity/Q754'), ('nm', 'Trinidad and Tobago')]
[('NumOfLeagues', '1'), ('o', 'http://www.wikidata.org/entity/Q854'), ('nm', 'Sri Lanka')]
[('NumOfLeagues', '3'), ('o', 'http://www.wikidata.org/entity/Q865'), ('nm', 'Taiwan')]
[('NumOfLeagues', '1'), ('o', 'http://www.wikidata.org/entity/Q921'), ('nm', '

In [116]:
#Selecying Croatia for getting continent
#[('NumOfLeagues', '4'), ('o', 'http://www.wikidata.org/entity/Q224'), ('nm', 'Croatia')]
queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
      wd:Q224 ?o ?p  .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?nm.
   filter(regex(?name,"Europe"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe'), ('o', 'http://www.wikidata.org/prop/direct/P30'), ('nm', 'continent')]
[('p', 'http://www.wikidata.org/entity/Q263686'), ('name', 'Southeast Europe'), ('o', 'http://www.wikidata.org/prop/direct/P361'), ('nm', 'part of')]
[('p', 'http://www.wikidata.org/entity/Q3866537'), ('name', 'Movement Coordination Centre Europe'), ('o', 'http://www.wikidata.org/prop/direct/P463'), ('nm', 'member of')]
[('p', 'http://www.wikidata.org/entity/Q81299'), ('name', 'Organization for Security and Co-operation in Europe'), ('o', 'http://www.wikidata.org/prop/direct/P463'), ('nm', 'member of')]
[('p', 'http://www.wikidata.org/entity/Q8908'), ('name', 'Council of Europe'), ('o', 'http://www.wikidata.org/prop/direct/P463'), ('nm', 'member of')]
[('p', 'http://www.wikidata.org/entity/Q458'), ('name', 'European Union'), ('o', 'http://www.wikidata.org/prop/direct/P463'), ('nm', 'member of')]
6


In [117]:
#filtering data according to continent
queryString = """
SELECT DISTINCT (count(?p) as ?NumOfLeagues) ?o ?nm
WHERE {
   # bind something
   ?p ?i wd:Q15991303.
   ?p wdt:P17 ?o.
   ?o wdt:P30 wd:Q46.
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?nm.
   
}
GROUP BY ?o ?nm
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfLeagues', '5'), ('o', 'http://www.wikidata.org/entity/Q28'), ('nm', 'Hungary')]
[('NumOfLeagues', '49'), ('o', 'http://www.wikidata.org/entity/Q218'), ('nm', 'Romania')]
[('NumOfLeagues', '3'), ('o', 'http://www.wikidata.org/entity/Q232'), ('nm', 'Kazakhstan')]
[('NumOfLeagues', '4'), ('o', 'http://www.wikidata.org/entity/Q215'), ('nm', 'Slovenia')]
[('NumOfLeagues', '3'), ('o', 'http://www.wikidata.org/entity/Q222'), ('nm', 'Albania')]
[('NumOfLeagues', '15'), ('o', 'http://www.wikidata.org/entity/Q38'), ('nm', 'Italy')]
[('NumOfLeagues', '3'), ('o', 'http://www.wikidata.org/entity/Q1246'), ('nm', 'Kosovo')]
[('NumOfLeagues', '93'), ('o', 'http://www.wikidata.org/entity/Q183'), ('nm', 'Germany')]
[('NumOfLeagues', '1'), ('o', 'http://www.wikidata.org/entity/Q9676'), ('nm', 'Isle of Man')]
[('NumOfLeagues', '6'), ('o', 'http://www.wikidata.org/entity/Q41'), ('nm', 'Greece')]
[('NumOfLeagues', '4'), ('o', 'http://www.wikidata.org/entity/Q189'), ('nm', 'Iceland')]
[('NumO

Final query for this task

In [135]:
#filtering data according to continent
#[('p', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe'), ('o', 'http://www.wikidata.org/prop/direct/P30'), ('nm', 'continent')]

queryString = """
SELECT DISTINCT (count(?p) as ?NumOfLeagues) ?o ?nm
WHERE {
   # bind something
   ?p ?i wd:Q15991303.
   ?p wdt:P17 ?o.
   ?o wdt:P30 wd:Q46.
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?nm.
   
}
GROUP BY ?o ?nm
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfLeagues', '5'), ('o', 'http://www.wikidata.org/entity/Q28'), ('nm', 'Hungary')]
[('NumOfLeagues', '49'), ('o', 'http://www.wikidata.org/entity/Q218'), ('nm', 'Romania')]
[('NumOfLeagues', '3'), ('o', 'http://www.wikidata.org/entity/Q232'), ('nm', 'Kazakhstan')]
[('NumOfLeagues', '4'), ('o', 'http://www.wikidata.org/entity/Q215'), ('nm', 'Slovenia')]
[('NumOfLeagues', '3'), ('o', 'http://www.wikidata.org/entity/Q222'), ('nm', 'Albania')]
[('NumOfLeagues', '15'), ('o', 'http://www.wikidata.org/entity/Q38'), ('nm', 'Italy')]
[('NumOfLeagues', '3'), ('o', 'http://www.wikidata.org/entity/Q1246'), ('nm', 'Kosovo')]
[('NumOfLeagues', '93'), ('o', 'http://www.wikidata.org/entity/Q183'), ('nm', 'Germany')]
[('NumOfLeagues', '1'), ('o', 'http://www.wikidata.org/entity/Q9676'), ('nm', 'Isle of Man')]
[('NumOfLeagues', '6'), ('o', 'http://www.wikidata.org/entity/Q41'), ('nm', 'Greece')]
[('NumOfLeagues', '4'), ('o', 'http://www.wikidata.org/entity/Q189'), ('nm', 'Iceland')]
[('NumO

## Task 5
Identify the BGP for the ballon d'Or award

In [125]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?o  .
   # get the label
   ?p sc:name ?name.
   filter(regex(?name,"award"))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
1


Final query for this task

In [136]:
# here are the awards recieved by cristiano ronaldo
# write your queries
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 wdt:P166 ?p.
   # get the label
   ?p sc:name ?name.
   #filter(regex(?name,"award"))
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1750244'), ('name', 'FWA Footballer of the Year')]
[('p', 'http://www.wikidata.org/entity/Q10726649'), ('name', "PFA Fans' Player of the Year")]
[('p', 'http://www.wikidata.org/entity/Q20888805'), ('name', 'Commander of the Order of Merit of Portugal')]
[('p', 'http://www.wikidata.org/entity/Q1146578'), ('name', 'UEFA Team of the Year')]
[('p', 'http://www.wikidata.org/entity/Q182529'), ('name', 'FIFA World Player of the Year')]
[('p', 'http://www.wikidata.org/entity/Q260117'), ('name', "UEFA Men's Player of the Year Award")]
[('p', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
[('p', 'http://www.wikidata.org/entity/Q20860117'), ('name', 'Officer of the Order of Prince Henry')]
[('p', 'http://www.wikidata.org/entity/Q20859673'), ('name', 'Grand Officer of the Order of Prince Henry')]
[('p', 'http://www.wikidata.org/entity/Q2986505'), ('name', 'Sir Matt Busby Player of the Year')]
[('p', 'http://www.wikidata.org/entit

In [129]:
# FINAL query 
# BGP OF BALLON D'OR 
#[('p', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q166177 ?p ?O.
   # get the label
   ?p sc:name ?name.
   #filter(regex(?name,"award"))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2572'), ('name', 'hashtag')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2924'), ('name', 'Great Russian Encyclopedia Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3222'), ('name', 'NE.se ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P64

## Task 6
Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

In [146]:
#[('p', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36386 ?O ?P.
   # get the label
   ?p sc:name ?name.
   
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P348'), ('name', 'software version identifier')]
[('p', 'http://www.wikidata.org/entity/P4669'), ('name', 'tabular software version')]
[('p', 'http://www.wikidata.org/entity/P747'), ('name', 'has edition or translation')]
[('p', 'http://www.wikidata.org/entity/Q1002728'), ('name', 'Fonsi Nieto')]
[('p', 'http://www.wikidata.org/entity/Q100326534'), ('name', "Al Stewart's albums in chronological order")]
[('p', 'http://www.wikidata.org/entity/Q100327667'), ('name', 'Category:Adelina Lopes Vieira')]
[('p', 'http://www.wikidata.org/entity/Q100343342'), ('name', 'Lubomír Stoklásek')]
[('p', 'http://www.wikidata.org/entity/Q1007576'), ('name', 'Ciudad Vieja')]
[('p', 'http://www.wikidata.org/entity/Q100879156'), ('name', 'Echániz')]
[('p', 'http://www.wikidata.org/entity/Q10100'), ('name', 'Natuna')]
[('p', 'http://www.wikidata.org/entity/Q101032298'), ('name', "Acid Mothers Temple's albums in chronological order")]
[('p', 'http://www.wikidata.

Final query for this task

In [ ]:
# write your final query

## Task 7
Identify the BGP of the FIFA World Cup

In [160]:
# [('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
#[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]

queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
      wd:Q170645 ?o ?p  .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?nm.
   #filter(regex(?nm,"league"))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q47774'), ('name', 'France national association football team'), ('o', 'http://www.wikidata.org/prop/direct/P1346'), ('nm', 'winner')]
[('p', 'http://www.wikidata.org/entity/Q19593927'), ('name', 'Template:2018 FIFA World Cup'), ('o', 'http://www.wikidata.org/prop/direct/P1424'), ('nm', "topic's main template")]
[('p', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia'), ('o', 'http://www.wikidata.org/prop/direct/P17'), ('nm', 'country')]
[('p', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football"), ('o', 'http://www.wikidata.org/prop/direct/P2094'), ('nm', 'competition class')]
[('p', 'http://www.wikidata.org/entity/Q202163'), ('name', 'Luzhniki Stadium'), ('o', 'http://www.wikidata.org/prop/direct/P276'), ('nm', 'location')]
[('p', 'http://www.wikidata.org/entity/Q1190634'), ('name', 'Otkrytie Bank Arena'), ('o', 'http://www.wikidata.org/prop/direct/P276'), ('nm', 'location')]
[('p', 'http://www.wikid

Final query for this task

In [165]:
# FINAL QUERY BGP FOR FIFA WORLD CUP
# [('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup'),
queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
      wd:Q19317 ?o ?p  .
   # get the label
   ?p sc:name ?name.
  
   
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q101751'), ('name', '1994 FIFA World Cup'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q131591'), ('name', '1938 FIFA World Cup'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q132664'), ('name', '1970 FIFA World Cup'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q253414'), ('name', 'FIFA'), ('o', 'http://www.wikidata.org/prop/direct/P664')]
[('p', 'http://www.wikidata.org/entity/Q48432'), ('name', '1930 FIFA World Cup'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q8437937'), ('name', 'Category:FIFA World Cup'), ('o', 'http://www.wikidata.org/prop/direct/P910')]
[('p', 'http://www.wikidata.org/entity/Q120'), ('name', 'June'), ('o', 'http://www.wikidata.org/prop/direct/P2922')]
[('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup'),

## Task 8
Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

In [172]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
      wd:Q19317 wdt:P527 ?p  .
   # get the label
   ?p sc:name ?name.
   #?o sc:name ?nm.
  
   
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q101751'), ('name', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q131591'), ('name', '1938 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q132664'), ('name', '1970 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q48432'), ('name', '1930 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q37285'), ('name', '2006 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q46934'), ('name', '1982 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q46938'), ('name', '1986 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q46943'), ('name', '1978 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q47735'), ('name', '2002 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q79859'), ('name', '2014 FIFA World Cup')]
[('

Final query for this task

In [ ]:
# write your final query

## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query